# 一、数据清洗

In [1]:
import pymongo
# MongoDB连接配置
mongo_client = pymongo.MongoClient("mongodb://localhost:27017/")
convoinsight_db = mongo_client["convoinsight-danghuan"]
conversations_collection = convoinsight_db["conversations"]

## 1. 清洗会话用户情感

In [5]:
#查看用户会话的sentiments的所有情感类型值和频数
# conversations_collection.find_one()
# 查询所有用户消息
from collections import Counter
user_messages = conversations_collection.aggregate([
    {"$unwind": "$messages"},
    {"$match": {"messages.type": "user"}},
    {"$project": {"sentiment": "$messages.sentiment"}}
])

# 提取情感并统计频数
sentiments = [msg['sentiment'] for msg in user_messages if 'sentiment' in msg]
sentiment_counts = Counter(sentiments)

# 输出结果
print(sentiment_counts)

Counter({'中立': 5730, '正向': 971, '负向': 882})


In [3]:
# 定义情感映射
sentiment_mapping = {
    '中立': '中立', 'neutral': '中立', 'Neutral': '中立', 'N/A': '中立', '': '中立',
    '负向': '负向', 'negative': '负向', 'Negative': '负向', 'negaitve': '负向', '負向': '负向',
    '正向': '正向', 'positive': '正向', 'Positive': '正向', '积极': '正向',
    # 其他情感类型可以根据需要进行归类
    'frustrated': '负向', 'concerned': '负向', 'disappointed': '负向',
    'apologetic': '负向', 'confused': '中立', '请求': '中立', '要求': '中立',
    'hopeful': '正向', '幽默': '中立', '疑惑': '中立', '疑问': '中立', 'affirmative': '正向'
}

# 更新数据库中的情感数据
for message in conversations_collection.find({"messages.type": "user"}):
    updated_messages = []
    for msg in message['messages']:
        if msg['type'] == 'user' and 'sentiment' in msg:
            original_sentiment = msg['sentiment']
            # 映射情感
            msg['sentiment'] = sentiment_mapping.get(original_sentiment, '中立')
        updated_messages.append(msg)
    
    # 更新数据库
    conversations_collection.update_one(
        {"_id": message["_id"]},
        {"$set": {"messages": updated_messages}}
    )


In [4]:
conversations_collection.find_one()

{'_id': ObjectId('6879b5f5d9eab0e645a9c2bf'),
 'agent': 'coco',
 'customerInfo': {'userId': '0630-7992', 'device': '旧手机', 'history': '无'},
 'interactionAnalysis': {'totalMessages': 5,
  'agentMessages': 3,
  'userMessages': 2},
 'messages': [{'type': 'user',
   'content': '我这个旧手机可以再贵一点吗',
   'sender': '0630-7992',
   'sentiment': '中立'},
  {'type': 'agent',
   'content': '您好呀！我是小当回收客服coco，很高兴能为您服务！',
   'sender': 'coco',
   'sentiment': None},
  {'type': 'agent',
   'content': '理解您的心情，二手手机行业与新机销售的市场类型不同，价格体系不同。',
   'sender': 'coco',
   'sentiment': None},
  {'type': 'agent',
   'content': '如果是已经提交了回收订单，辛苦提供回收订单号或者下单的手机号码，客服为您查询',
   'sender': 'coco',
   'sentiment': None}],
 'origin_conversation': '江苏/连云港0630-7992 2025-06-30 19:57:33_x000D_\n我这个旧手机可以再贵一点吗_x000D_\n_x000D_\ncoco 2025-06-30 19:57:42_x000D_\n您好呀！我是小当回收客服coco，很高兴能为您服务！_x000D_\n_x000D_\ncoco 2025-06-30 19:57:49_x000D_\n理解您的心情，二手手机行业与新机销售的市场类型不同，价格体系不同。_x000D_\n_x000D_\ncoco 2025-06-30 19:58:01_x000D_\n如果是已经提交了回收订单，辛苦提供回收订单号或